In [1]:
import os
#Importo lo necesario para spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import length, col,udf,row_number,expr,when,from_unixtime
from functools import reduce
from pyspark.sql.types import TimestampType
import pandas as pd

# inicio la sesion de spark
spark = SparkSession.builder \
    .appName("TuAppName") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "30g") \
    .config("spark.network.timeout", "1000s") \
    .config("spark.dynamicAllocation.maxExecutors", "4") \
    .master("local[*]") \
    .getOrCreate()

23/11/23 12:33:48 WARN Utils: Your hostname, DESKTOP-FFOU6NQ resolves to a loopback address: 127.0.1.1; using 192.168.0.20 instead (on interface eth0)
23/11/23 12:33:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/23 12:33:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Google Maps

In [2]:
#lista con nombre de los estados
estados = []
ruta = '/mnt/c/Henry_PF/GP_Henry/datasets/Google Maps/reviews-estados'
for estado in os.listdir(ruta):
        estado_path = os.path.join(ruta, estado)
        
        if os.path.isdir(estado_path) and estado.startswith('review-'):
            nombre_estado = estado[len('review-'):]
            estados.append(nombre_estado)

In [3]:
# Funcion para leer todos los json de uina carpeta y unirlos en un datafram
def read_all_json(folder_ruth):
    rutas = []
    for archivo_json in os.listdir(folder_ruth):
        if archivo_json.endswith('.json'):
            # Construye la ruta completa del archivo JSON
            ruta_archivo_json = os.path.join(folder_ruth, archivo_json)
            
            # Lee el archivo JSON en un DataFrame
            #df = spark.read.json(ruta_archivo_json)
            
            # Agrega el DataFrame a la lista
            rutas.append(ruta_archivo_json)

    # Concatena todos los DataFrames en uno solo
    dataframes = [spark.read.json(file_path) for file_path in rutas]
    merged_dataframe = reduce(lambda df1, df2: df1.union(df2), dataframes)
    return merged_dataframe

In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
def analyze_sentiment(text):
    if text is None:
        return 0
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(text)["compound"]
    if sentiment_score > 0.3:
        return 2
    elif sentiment_score<-0.3:
        return 1

# Aplico la funcion al dataframe
from pyspark.sql.types import StringType
analyze_sentiment_udf = udf(analyze_sentiment, StringType())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/bruno/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/bruno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:

    
#Trasnformo los foramtos de time y extraigo time y text de resp
def format_time_second(milliseconds):
    # Convertir milisegundos a segundos dividiendo por 1000
    seconds = milliseconds / 1000.0
    
    # Utilizar from_unixtime con el formato deseado, incluyendo milisegundos
    return from_unixtime(seconds, "yyyy-MM-dd HH:mm:ss.SSS")
def etl_auto(df):
    df = df.withColumn("time", format_time_second(col("time")))
    df = df.withColumn("resp_text", col("resp").getItem("text"))
    df = df.withColumn("resp_time", col("resp").getItem("time"))
    df = df.withColumn("resp_time", format_time_second(col("resp_time")))
    
    df = df.withColumn("sentiment", analyze_sentiment_udf(col("text")))
    df = df.withColumn("sentiment", when(col("text").isNull(), 0).otherwise(col("sentiment")))
    df = df.withColumn("resp_sentiment", analyze_sentiment_udf(col("resp_text")))
    df = df.withColumn("resp_sentiment", when(col("resp_text").isNull(), 0).otherwise(col("resp_sentiment")))
    
    # Elimina la columna 'resp'
    df = df.drop('resp','text','resp_text')
    
    return df



In [6]:
#Concateno los json de cada estado en uno solo y lo exporto a parquet
for estado in estados:
    rutas = f'{ruta}/review-{estado}'
    df = read_all_json(rutas)
    df = etl_auto(df)
    df_pandas = df.toPandas()
    df.write.parquet(f'{ruta}/datalake-estados/all_{estado}.parquet', mode='overwrite',compression='gzip')

ERROR:root:KeyboardInterrupt while sending command.                (8 + 8) / 96]
Traceback (most recent call last):
  File "/home/bruno/Henry_PF/spark-3.5.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/bruno/Henry_PF/spark-3.5.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 